<a href="https://colab.research.google.com/github/ananyasgit/Passive-Agressive-Classifier/blob/main/FakeNewsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.util import ngrams
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Reading the dataset

In [2]:
news_df=pd.read_csv("news_dataset.csv", engine='python',encoding='utf-8', error_bad_lines=False)
news_df


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,label,text
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...
1,FAKE,A four-minute-long video of a woman criticisin...
2,FAKE,"Republic Poll, a fake Twitter account imitatin..."
3,REAL,"Delhi teen finds place on UN green list, turns..."
4,REAL,Delhi: A high-level meeting underway at reside...
...,...,...
3724,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3725,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3726,FAKE,The Bengaluru City Police’s official Twitter h...
3727,REAL,"Sep 20, 2020, 08:00AM IST\n\nSource: TOI.in\n\..."


In [3]:
news_text=news_df['text']
news_text

0       Payal has accused filmmaker Anurag Kashyap of ...
1       A four-minute-long video of a woman criticisin...
2       Republic Poll, a fake Twitter account imitatin...
3       Delhi teen finds place on UN green list, turns...
4       Delhi: A high-level meeting underway at reside...
                              ...                        
3724    19:17 (IST) Sep 20\n\nThe second round of coun...
3725    19:17 (IST) Sep 20\n\nThe second round of coun...
3726    The Bengaluru City Police’s official Twitter h...
3727    Sep 20, 2020, 08:00AM IST\n\nSource: TOI.in\n\...
3728    Read Also\n\nRead Also\n\nAdvocate Ishkaran Bh...
Name: text, Length: 3729, dtype: object

Cleaning the text

In [4]:
news_df['text']=news_df['text'].apply(str)

In [5]:
news_df['text']= news_df['text'].apply((lambda y:re.sub("http://\S+"," ", y)))
news_df['text']= news_df['text'].apply((lambda x:re.sub("\@", " ",x.lower())))
news_df['text']= news_df['text'].apply((lambda z:re.sub(r'[^\w\s]', '', z)))
news_df['text']= news_df['text'].apply((lambda w:re.sub("[0-9]", "", w)))
news_df['text']= news_df['text'].apply((lambda v:re.sub("\n", "", v)))

news_df['text']

0       payal has accused filmmaker anurag kashyap of ...
1       a fourminutelong video of a woman criticising ...
2       republic poll a fake twitter account imitating...
3       delhi teen finds place on un green list turns ...
4       delhi a highlevel meeting underway at residenc...
                              ...                        
3724     ist sep the second round of countrywide seros...
3725     ist sep the second round of countrywide seros...
3726    the bengaluru city polices official twitter ha...
3727    sep   am istsource toiinmeet neelkantha bhanu ...
3728    read alsoread alsoadvocate ishkaran bhandari r...
Name: text, Length: 3729, dtype: object

In [6]:
news_df['text'][1]

'a fourminutelong video of a woman criticising the government at an anticitizenship amendment act rally in delhi earlier in january is going viral with a false claim that the woman shows late prime minister atal bihari vajpayees niecethe caption in hindi translates to  respected vajpayee jis niece has finally broken her silence listen to what she has to saytranslated from hindi मननय वजपय ज क भतज न आखरकर तड चपप  जनए कय कहalso read muslim politician disguised as a hindu in anticaa proteststhe video comes at a time when protests against the citizenship amendment act and a proposed national register of citizens have gone unabated for over a monthin the video the woman can be seen saying the following in hindithey britishers did a lot of bad but at least they were outsiders first of all they were not ours they were not from this land they came from afar and came here to do this but still the difference between them indian government and britishers was that they britishers were educated they

In [7]:
#For removing hindi characters from the dataset
news_df['text'] = news_df['text'].apply(lambda q:re.sub("([^\x900-\x97F])+"," ",q))

In [8]:
news_df['text'][1] #hindi characters successfully removed 

'a fourminutelong video of a woman criticising the government at an anticitizenship amendment act rally in delhi earlier in january is going viral with a false claim that the woman shows late prime minister atal bihari vajpayees niecethe caption in hindi translates to respected vajpayee jis niece has finally broken her silence listen to what she has to saytranslated from hindi also read muslim politician disguised as a hindu in anticaa proteststhe video comes at a time when protests against the citizenship amendment act and a proposed national register of citizens have gone unabated for over a monthin the video the woman can be seen saying the following in hindithey britishers did a lot of bad but at least they were outsiders first of all they were not ours they were not from this land they came from afar and came here to do this but still the difference between them indian government and britishers was that they britishers were educated they were not this illiterate at least they didn

In [9]:
stop = stopwords.words('english')
news_df['text']= news_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
news_df['text']


0       payal accused filmmaker anurag kashyap behavin...
1       fourminutelong video woman criticising governm...
2       republic poll fake twitter account imitating a...
3       delhi teen finds place un green list turns gla...
4       delhi highlevel meeting underway residence raj...
                              ...                        
3724    ist sep second round countrywide serosurvey le...
3725    ist sep second round countrywide serosurvey le...
3726    bengaluru city polices official twitter handle...
3727    sep istsource toiinmeet neelkantha bhanu praka...
3728    read alsoread alsoadvocate ishkaran bhandari r...
Name: text, Length: 3729, dtype: object

In [10]:
#print(news_df.text.apply(word_tokenize))

In [11]:
ps = PorterStemmer()
corpus = []
review = [ps.stem(word) for word in news_df['text']if not word in stopwords.words('english')]
review = ' '.join(review)
corpus.append(review)

In [12]:
labels=news_df.label
labels.head()

0    REAL
1    FAKE
2    FAKE
3    REAL
4    REAL
Name: label, dtype: object

Training and testing

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(news_df['text'], labels, test_size=0.2, random_state=7)

In [14]:
#initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [15]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
pac=PassiveAggressiveClassifier(max_iter=100)
pac.fit(tfidf_train,y_train)
#Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 99.73%


In [16]:
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[357,   1],
       [  1, 387]])

In [17]:
# Tokenization
review = re.sub('[^a-zA-Z]', ' ', news_df['text'][1])
review = review.lower()
review = review.split() 
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
# Vectorization 
val = tfidf_vectorizer.transform([review]).toarray()
# Predict 
pac.predict(val)

array(['FAKE'], dtype='<U4')

Pickling the model

In [18]:
import pickle
pickle.dump(pac, open('model.pkl', 'wb'))
pickle.dump(tfidf_vectorizer, open('tfidfvect.pkl', 'wb'))

In [19]:
joblib_model = pickle.load(open('model.pkl', 'rb'))
joblib_vect = pickle.load(open('tfidfvect.pkl', 'rb'))
val_pkl = joblib_vect.transform([review]).toarray()
joblib_model.predict(val_pkl)

array(['FAKE'], dtype='<U4')